In [2]:
import json

car_group = dict()


k5 = dict()

k5["price"] = "5000"

k5["year"] = "2015"

car_group["K5"] = k5


avante = dict()

avante["price"] = "3000"

avante["year"] = "2014"

car_group["Avante"] = avante


#json 파일로 저장

with open('./test.json', 'w', encoding='utf-8') as make_file:

    json.dump(car_group, make_file, indent="\t")


# 저장한 파일 출력하기

with open('./test.json', 'r') as f:

    json_data = json.load(f)

print(json.dumps(json_data, indent="\t"))


{
	"K5": {
		"price": "5000",
		"year": "2015"
	},
	"Avante": {
		"price": "3000",
		"year": "2014"
	}
}


In [10]:
import fitz
import PIL.Image
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar,LTLine,LAParams
import os
import json


path = r'./os.pdf'

extract_data = {}

for page_layout in extract_pages(path):
    data = []
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            for text_line in element:
                for character in text_line:
                    if isinstance(character, LTChar):
                        Font_size = round(character.size)
            data.append({'size': Font_size, 'text': (element.get_text())})
    extract_data[page_layout.pageid] = data

with open('./text.json', 'w', encoding='utf-8') as make_file:

    json.dump(extract_data, make_file, indent="\t", ensure_ascii=False)
print("fin")


fin


In [2]:
import pdfminer
from pdfminer.image import ImageWriter
from pdfminer.high_level import extract_pages

pages = list(extract_pages('os.pdf'))
page = pages[0]


def get_image(layout_object):
    if isinstance(layout_object, pdfminer.layout.LTImage):
        return layout_object
    if isinstance(layout_object, pdfminer.layout.LTContainer):
        for child in layout_object:
            return get_image(child)
    else:
        return None


def save_images_from_page(page: pdfminer.layout.LTPage):
    images = list(filter(bool, map(get_image, page)))
    iw = ImageWriter('output_dir')
    for image in images:
        iw.export_image(image)


save_images_from_page(page)

